In [4]:
%%writefile app.py
import os
import pandas as pd
import numpy as np
from flask import Flask, flash, request, redirect, url_for, send_from_directory, render_template
from werkzeug.utils import secure_filename
import sqlite3
import query_alerts
from sqlalchemy import create_engine, text
from sqlalchemy.sql import select
import datetime

app = Flask(__name__)

# Write a function to view lightcurves (click and can go back to list, or go "next"), with an option to sort.
# Write a function to query and sort. 
# Option to save querys? 
# How to do version control? (As alert values update.) Have multiple databases? 
# Save in the database name the day it was created? 
# Have some function that knows the current year. If it was an old year, this is unnecessary.

engine = create_engine('sqlite:///microlensing.db')
conn = engine.connect()

@app.route('/', methods=['GET', 'POST'])
def start_page():
    """
    Homepage. Depending on whether there is a database or not,
    will either prompt you to upload, or will give you the 
    option to upload or query.
    """
    if engine.table_names() == []:
        return render_template('start_empty.html', web_download_to_db=url_for('web_download_to_db'))

    else:
        return render_template('start_filled.html', 
                               web_download_to_db=url_for('web_download_to_db'), 
                               query_db=url_for('query_db'),
                               dbs=engine.table_names())

@app.route('/update', methods=['GET', 'POST'])
def web_download_to_db():
    """
    FIXME: add an option to choose whether to keep or delete database.
    Don't redownload old years?
    """
    
    return render_template('download_data.html', start_page=url_for('start_page'))
    # return render_template('example.html')

@app.route('/handle_data', methods=['GET', 'POST'])
def handle_data():
    """
    FIXME: add an option to choose whether to keep or delete database.
    Actually, have radio buttons for this: Choose whether want lightcurves, data, system, and years.

    """
    duplicate_list = []
    update_list = []
    download_list = []

    this_year = str(datetime.date.today().year)
    
    keys = list(request.form.to_dict().keys())
    for ii, key in enumerate(keys):
        system, data, year = key.split('_')
        print(key)
        print(engine.table_names())
        
        if key in engine.table_names():
            if this_year == year:
                update_list.append(key)
                # FIXME: DELETE THE OLD TABLE FROM THE DATABASE.
            else:
                duplicate_list.append(key)
        else:
            download_list.append(key)

    for ii, key in enumerate(keys):
        system, data, year = key.split('_')
        if key in update_list + download_list:
            if system == 'kmtnet':
                if data == 'alerts':
                    query_alerts.get_kmtnet_alerts(year)
                else: # lightcurves
                    query_alerts.get_kmtnet_lightcurves(year)
            elif system == 'ogle':
                if data == 'alerts':
                    query_alerts.get_ogle_alerts(year)
                else: # lightcurves
                    query_alerts.get_ogle_lightcurves(year)
            else: # MOA
                if data == 'alerts':
                    query_alerts.get_moa_alerts(year)
                else: # lightcurves
                    query_alerts.get_moa_lightcurves(year)
    # FIXME: CHECK ON CAPITALIZATIONS AND THINGS!!!!!!!!!
    print('update_list', update_list)
    print('duplicate_list', duplicate_list)
    print('download_list', download_list)
    # How to show processing? Googling "stream" and "dynamic" but I don't think that's what I want.
    return render_template('download_results.html', 
                            duplicate_list=duplicate_list,
                            update_list=update_list,
                            download_list=download_list,
                            web_download_to_db=url_for('web_download_to_db'), 
                            query_db=url_for('query_db'),
                            start_page=url_for('start_page'))

@app.route('/query', methods=['GET', 'POST'])
def query_db():
    """
    Provide the interface to query the database.
    """
    if request.method == 'POST':
        query_str = request.form['query']
        db_info = engine.execute(query_str).fetchall()
        
        if len(db_info) == 0:
            return render_template('display_empty.html', 
                                   query_str=query_str,
                                   query_db=url_for('query_db'),
                                   start_page=url_for('start_page'))
        else:
            # db_info = [[str(x).ljust(20) for x in line] for line in db_info]
            return render_template('display.html', 
                                   query_str=query_str,
                                   len = len(db_info), 
                                   db_info = db_info, 
                                   start_page=url_for('start_page'))

    return render_template('query.html', 
                           web_download_to_db=url_for('web_download_to_db'),
                           start_page=url_for('start_page'), 
                           dbs=engine.table_names())

if __name__ == '__main__':
    app.run(port=8000, debug = True)

Overwriting app.py


In [ ]:
!python app.py

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 145-043-719
ogle_lightcurves_2019
/u/casey/scratch/code/python-ay250-homeworks/final_project/app.py:67: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(engine.table_names())
['kmtnet_2021', 'kmtnet_2022', 'kmtnet_alerts_2016', 'kmtnet_alerts_2017', 'kmtnet_alerts_2018', 'kmtnet_alerts_2019', 'kmtnet_alerts_2020', 'kmtnet_alerts_2021', 'kmtnet_alerts_2022', 'moa_2021', 'moa_2022', 'moa_alerts_2016', 'moa_alerts_2017', 'moa_alerts_2018', 'moa_alerts_2019', 'moa_alerts_2020', 'moa_alerts_2021', 'moa_alerts_2022', 'ogle_2011', 'ogle_2019', 'ogle_alerts_2011', 'ogle_alerts_2012', 'ogle_alerts_2013', 'ogle_a

blg-0604
blg-0605
blg-0606
blg-0607
blg-0608
blg-0609
blg-0610
blg-0611
blg-0612
blg-0613
blg-0614
blg-0615
blg-0616
blg-0617
blg-0618
blg-0619
blg-0620
blg-0621
blg-0622
blg-0623
blg-0624
blg-0625
blg-0626
blg-0627
blg-0628
blg-0629
blg-0630
blg-0631
blg-0632
blg-0633
blg-0634
blg-0635
blg-0636
blg-0637
blg-0638
blg-0639
blg-0640
blg-0641
blg-0642
blg-0643
blg-0644
blg-0645
blg-0646
blg-0647
blg-0648
blg-0649
blg-0650
blg-0651
blg-0652
blg-0653
blg-0654
blg-0655
blg-0656
blg-0657
blg-0658
blg-0659
blg-0660
blg-0661
blg-0662
blg-0663
blg-0664
blg-0665
blg-0666
blg-0667
blg-0668
blg-0669
blg-0670
blg-0671
blg-0672
blg-0673
blg-0674
blg-0675
blg-0676
blg-0677
blg-0678
blg-0679
blg-0680
blg-0681
blg-0682
blg-0683
blg-0684
blg-0685
blg-0686
blg-0687
blg-0688
blg-0689
blg-0690
blg-0691
blg-0692
blg-0693
blg-0694
blg-0695
blg-0696
blg-0697
blg-0698
blg-0699
blg-0700
blg-0701
blg-0702
blg-0703
blg-0704
blg-0705
blg-0706
blg-0707
blg-0708
blg-0709
blg-0710
blg-0711
blg-0712
blg-0713
blg-0714
b

blg-1515
blg-1516
blg-1517
blg-1518
blg-1519
blg-1520
blg-1521
blg-1522
blg-1523
blg-1524
blg-1525
blg-1526
Took 2556.68 seconds
update_list []
duplicate_list []
download_list ['ogle_lightcurves_2019']
127.0.0.1 - - [05/May/2022 11:07:06] "POST /handle_data HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 11:08:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 11:09:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 11:09:28] "POST /query HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1820, in _execute_context
    cursor, statement, parameters, context
  File "/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/sqlalchemy/engine/default.py", line 732, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: no such table: ogle_2019

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/an